In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import re

In [3]:
# get loandata
loansData = pd.read_csv('https://spark-public.s3.amazonaws.com/dataanalysis/loansData.csv')

In [4]:
# clean interest rate using regex - ability to replace so df objects are mutable
loansData['Interest.Rate'] = [float(re.search('^[0-9]*.[0-9]*',loan).group(0))/100 for loan in loansData['Interest.Rate']]

In [5]:
# clean fico range and get score 
loansData['FICO.Score'] = map(lambda x: int(x.split('-')[0]), loansData['FICO.Range'])

In [6]:
def rate_check(int_rate):
    if int_rate < .12:
        return 1
    else:
        return 0
    

In [7]:
loansData['ir_tf'] = [rate_check(int_rate) for int_rate in loansData['Interest.Rate']]

In [1]:
list(loansData.column.values)

NameError: name 'loansData' is not defined

In [9]:
loansData.to_csv('loansData_clean.csv', header=True, index=False)

In [10]:
loansData[loansData['Interest.Rate'] == .10].head() # should all be True

,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length,FICO.Score,ir_tf
22150,6000,5950,0.1,36 months,major_purchase,8.74%,FL,MORTGAGE,15416.67,715-719,9,105730,1,10+ years,715,1
23437,21000,20925,0.1,36 months,credit_card,14.08%,CA,RENT,6250.00,730-734,8,21102,0,2 years,730,1
19321,12000,12000,0.1,36 months,debt_consolidation,12.32%,WI,MORTGAGE,5000.00,715-719,6,4515,1,7 years,715,1
21900,2800,2800,0.1,36 months,debt_consolidation,20.32%,CA,RENT,3250.00,700-704,4,8717,0,5 years,700,1
23214,5000,5000,0.1,60 months,car,9.28%,FL,RENT,4000.00,735-739,6,3448,0,< 1 year,735,1


In [11]:
loansData['const'] = [1 for int_rate in loansData['Interest.Rate']]

In [17]:
type(loansData['const'].tolist()[0])


numpy.int64

In [18]:
ind_vars = list(loansData.columns.values)

In [20]:
ind_vars = ['Amount.Requested',
 'Amount.Funded.By.Investors',
 'Interest.Rate',
 'Loan.Length',
 'Loan.Purpose',
 'Debt.To.Income.Ratio',
 'State',
 'Home.Ownership',
 'Monthly.Income',
 'FICO.Range',
 'Open.CREDIT.Lines',
 'Revolving.CREDIT.Balance',
 'Inquiries.in.the.Last.6.Months',
 'Employment.Length',
 'FICO.Score',
 'const']

In [22]:
logit = sm.Logit(loansData['ir_tf'], loansData[ind_vars])

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [23]:
loansData.head()

,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length,FICO.Score,ir_tf,const
81174,20000,20000,0.0890,36 months,debt_consolidation,14.90%,SC,MORTGAGE,6541.67,735-739,14,14272,2,< 1 year,735,1,1
99592,19200,19200,0.1212,36 months,debt_consolidation,28.36%,TX,MORTGAGE,4583.33,715-719,12,11140,1,2 years,715,0,1
80059,35000,35000,0.2198,60 months,debt_consolidation,23.81%,CA,MORTGAGE,11500.00,690-694,14,21977,1,2 years,690,0,1
15825,10000,9975,0.0999,36 months,debt_consolidation,14.30%,KS,MORTGAGE,3833.33,695-699,10,9346,0,5 years,695,1,1
33182,12000,12000,0.1171,36 months,credit_card,18.78%,NJ,RENT,3195.00,695-699,11,14469,0,9 years,695,1,1


In [76]:
np.asarray(loansData)


array([[20000L, 20000.0, 0.08900000000000001, ..., 735L, 1L, 1L],
       [19200L, 19200.0, 0.12119999999999999, ..., 715L, 0L, 1L],
       [35000L, 35000.0, 0.2198, ..., 690L, 0L, 1L],
       ..., 
       [10000L, 10000.0, 0.1399, ..., 680L, 0L, 1L],
       [6000L, 6000.0, 0.1242, ..., 675L, 0L, 1L],
       [9000L, 5242.75, 0.1379, ..., 670L, 0L, 1L]], dtype=object)

In [87]:
[ind_vars[i] for i,strvalue in enumerate(ind_vars)]

['Amount.Requested',
 'Amount.Funded.By.Investors',
 'Interest.Rate',
 'Loan.Length',
 'Loan.Purpose',
 'Debt.To.Income.Ratio',
 'State',
 'Home.Ownership',
 'Monthly.Income',
 'FICO.Range',
 'Open.CREDIT.Lines',
 'Revolving.CREDIT.Balance',
 'Inquiries.in.the.Last.6.Months',
 'Employment.Length',
 'FICO.Score',
 'ir_tf',
 'const']

In [89]:
type(loansData['const'].head())

pandas.core.series.Series